In [37]:
# 配置文件读取 yaml
import yaml

# 四元数运算库
import quaternion # qw qx qy qz

# math 
import math

### matplotlib
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['text.usetex'] = True # 在图片中使用 latex 
import scienceplots

### pytorch 相关
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

### numpy
import numpy as np

### SciPy
import scipy.signal as signal

# jupyter 使用 matplotlib 绘图所需，否则会挂掉
import os 
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# 文件操作
import shutil

# 进度条
from tqdm import tqdm

# 随机
import random

# time
import time

In [38]:
class EurocAnalyser(object):

    def __init__(self, project_path, data_set):
        self.project_path = project_path
        self.data_set = data_set

    def path_generator(self):
        # 找到配置文件
        configs_path = os.path.join(self.project_path, 'configs/euroc_configs.yaml')
        assert os.path.isfile(configs_path)
        # 读取配置文件
        with open(configs_path, 'r', encoding='utf8') as file:
            configs = yaml.safe_load(file)
        self.relative_data_path = 'data/Euroc/' + self.data_set
        # 生成 gt_file 和 imu_file 文件的路径
        self.abs_data_path = os.path.join(self.project_path, self.relative_data_path)
        self.gt_file_path = os.path.join(self.abs_data_path, 'state_groundtruth_estimate0/data.csv')
        self.imu_file_path = os.path.join(self.abs_data_path, 'imu0/data.csv')
        # 判断几个文件是否存在
        assert os.path.isfile(self.gt_file_path)
        assert os.path.isfile(self.imu_file_path)
        # 返回相应的路径
        return self.gt_file_path, self.imu_file_path

    def get_size_set_list(self):
        # 找到配置文件
        configs_path = os.path.join(self.project_path, 'configs/euroc_configs.yaml')
        assert os.path.isfile(configs_path)
        # 读取配置文件
        with open(configs_path, 'r', encoding='utf8') as file:
            configs = yaml.safe_load(file)
        self.size_set_list = configs['size_set_list']
        assert len(self.size_set_list) >= 1
        return self.size_set_list

    def get_gt_pose(self):
        with open(self.gt_file_path, 'r', encoding = 'utf-8') as file_gt:
            data = np.loadtxt(file_gt, delimiter=',', skiprows=0)
        self.gt_timestamp = data[:,0] # 此处时间单位为 ns !!!!!
        self.gt_px_m = data[:,1]
        self.gt_py_m = data[:,2]
        self.gt_pz_m = data[:,3]
        self.gt_qw = data[:,4]
        self.gt_qx = data[:,5]
        self.gt_qy = data[:,6]
        self.gt_qz = data[:,7]
        self.gt_vx_mps = data[:,8]
        self.gt_vy_mps = data[:,9]
        self.gt_vz_mps = data[:,10]

    def get_imu_raw_data(self):
        with open(self.imu_file_path, 'r', encoding = 'utf-8') as file_imu_raw:
            data = np.loadtxt(file_imu_raw, delimiter=',', skiprows=0)
        self.imu_raw_timestamp = data[:,0]  # 此处时间单位为 ns !!!!!
        self.imu_raw_wx = data[:,1]
        self.imu_raw_wy = data[:,2]
        self.imu_raw_wz = data[:,3]
        self.imu_raw_ax = data[:,4]
        self.imu_raw_ay = data[:,5]
        self.imu_raw_az = data[:,6]

    def timestamp_align_v2(self):
        ### 时间戳完全对齐，以imu时刻为基准，调整gt相对应时刻的值（线性插值）
        #### 新建空白数组 时间戳
        self.aligned_timestamp = np.array([])
        #### 新建空白数据 IMU 数据
        self.aligned_imu_wx = np.array([])
        self.aligned_imu_wy = np.array([])
        self.aligned_imu_wz = np.array([])
        self.aligned_imu_ax = np.array([])
        self.aligned_imu_ay = np.array([])
        self.aligned_imu_az = np.array([])
        #### 新建空白数组 位移
        self.aligned_px_m = np.array([])
        self.aligned_py_m = np.array([])
        self.aligned_pz_m = np.array([])
        #### 新建空白数组 速度
        self.aligned_vx_mps = np.array([])
        self.aligned_vy_mps = np.array([])
        self.aligned_vz_mps = np.array([])
        #### 新建空白数组 姿态
        self.aligned_qw = np.array([])
        self.aligned_qx = np.array([])
        self.aligned_qy = np.array([])
        self.aligned_qz = np.array([])

        i = 0 # idx for imu, 以imu时刻为基准
        j = 0 # idx for gt

        while i < np.size(self.imu_raw_timestamp)-1 and j < np.size(self.gt_timestamp)-1:
            detT_threshold = 2500000
            if (self.gt_timestamp[j] - self.imu_raw_timestamp[i]) > detT_threshold:
                # print(self.gt_timestamp[j] - self.imu_raw_timestamp[i])
                i = i + 1
                continue
            if (self.imu_raw_timestamp[i] - self.gt_timestamp[j]) > detT_threshold:
                # print(self.imu_raw_timestamp[i] - self.gt_timestamp[j])
                j = j + 1
                continue
            
            # time.sleep(1)
            # print(self.imu_raw_timestamp[i] - self.gt_timestamp[j])
            assert abs(self.imu_raw_timestamp[i] - self.gt_timestamp[j]) < 1000000

            # 时间戳
            self.aligned_timestamp = np.append(self.aligned_timestamp, self.imu_raw_timestamp[i])
            # 位移插值
            self.aligned_px_m = np.append(self.aligned_px_m, self.gt_px_m[j])
            self.aligned_py_m = np.append(self.aligned_py_m, self.gt_py_m[j])
            self.aligned_pz_m = np.append(self.aligned_pz_m, self.gt_pz_m[j])
            # 四元数插值
            self.aligned_qw = np.append(self.aligned_qw, self.gt_qw[j])
            self.aligned_qx = np.append(self.aligned_qx, self.gt_qx[j])
            self.aligned_qy = np.append(self.aligned_qy, self.gt_qy[j])
            self.aligned_qz = np.append(self.aligned_qz, self.gt_qz[j])
            # IMU测量数据
            self.aligned_imu_wx = np.append(self.aligned_imu_wx, self.imu_raw_wx[i])
            self.aligned_imu_wy = np.append(self.aligned_imu_wy, self.imu_raw_wy[i])
            self.aligned_imu_wz = np.append(self.aligned_imu_wz, self.imu_raw_wz[i])
            self.aligned_imu_ax = np.append(self.aligned_imu_ax, self.imu_raw_ax[i])
            self.aligned_imu_ay = np.append(self.aligned_imu_ay, self.imu_raw_ay[i])
            self.aligned_imu_az = np.append(self.aligned_imu_az, self.imu_raw_az[i])
            # 此处不对 j 进行叠加，原因：两个gt时刻间可能有多个imu测量值
            i = i + 1
            j = j + 1
        # 将 时间戳的单位由纳秒转为秒
        self.aligned_timestamp = self.aligned_timestamp * 1e-9

    def timestamp_align_old(self):
        ### 时间戳完全对齐，以imu时刻为基准，调整gt相对应时刻的值（线性插值）
        #### 新建空白数组 时间戳
        self.aligned_timestamp = np.array([])
        #### 新建空白数据 IMU 数据
        self.aligned_imu_wx = np.array([])
        self.aligned_imu_wy = np.array([])
        self.aligned_imu_wz = np.array([])
        self.aligned_imu_ax = np.array([])
        self.aligned_imu_ay = np.array([])
        self.aligned_imu_az = np.array([])
        #### 新建空白数组 位移
        self.aligned_px_m = np.array([])
        self.aligned_py_m = np.array([])
        self.aligned_pz_m = np.array([])
        #### 新建空白数组 速度
        self.aligned_vx_mps = np.array([])
        self.aligned_vy_mps = np.array([])
        self.aligned_vz_mps = np.array([])
        #### 新建空白数组 姿态
        self.aligned_qw = np.array([])
        self.aligned_qx = np.array([])
        self.aligned_qy = np.array([])
        self.aligned_qz = np.array([])

        i = 0 # idx for imu, 以imu时刻为基准
        j = 0 # idx for gt

        while i < np.size(self.imu_raw_timestamp)-1 and j < np.size(self.gt_timestamp)-1:
            # imu_i = self.imu_raw_timestamp[i]
            # self.gt_j = self.gt_timestamp[j]
            # self.gt_j1 = self.gt_timestamp[j+1]
            # imu的时间戳早于gt，说明imu启动的时间早于gt，即当前的imu时刻无法获取真实位姿
            # imu_i ... --> self.gt_j
            if self.imu_raw_timestamp[i] < self.gt_timestamp[j]:
                i = i + 1
                continue
            # imu的时间戳晚于gt多次
            # self.gt_j -->  self.gt_j+1 --> ... --> imu_i
            if self.imu_raw_timestamp[i] > self.gt_timestamp[j] and self.imu_raw_timestamp[i] > self.gt_timestamp[j+1]:
                j = j + 1
                continue
            # imu的时间戳晚于gt一次
            # self.gt_j -->  imu_i --> self.gt_j+1
            if self.imu_raw_timestamp[i] >= self.gt_timestamp[j] and self.imu_raw_timestamp[i] <= self.gt_timestamp[j+1]:
                # 插值比例
                fraction = (self.imu_raw_timestamp[i] - self.gt_timestamp[j])/(self.gt_timestamp[j+1] - self.gt_timestamp[j])
                # print(self.imu_raw_timestamp[i] - self.gt_timestamp[j])
                # time.sleep(4)
                # 时间戳
                self.aligned_timestamp = np.append(self.aligned_timestamp, self.imu_raw_timestamp[i])
                # 位移插值
                tmp_px_m = self.gt_px_m[j] + (self.gt_px_m[j+1] - self.gt_px_m[j])*fraction
                self.aligned_px_m = np.append(self.aligned_px_m, tmp_px_m)
                tmp_py_m = self.gt_py_m[j] + (self.gt_py_m[j+1] - self.gt_py_m[j])*fraction
                self.aligned_py_m = np.append(self.aligned_py_m, tmp_py_m)
                tmp_pz_m = self.gt_pz_m[j] + (self.gt_pz_m[j+1] - self.gt_pz_m[j])*fraction
                self.aligned_pz_m = np.append(self.aligned_pz_m, tmp_pz_m)
                # 四元数插值
                q_j = np.quaternion(self.gt_qw[j], self.gt_qx[j], self.gt_qy[j], self.gt_qz[j])
                q_j1 = np.quaternion(self.gt_qw[j+1], self.gt_qx[j+1], self.gt_qy[j+1], self.gt_qz[j+1])
                # q_i = tf.transformations.quaternion_slerp(q_j, q_j1, fraction)
                q_i = quaternion.slerp_evaluate(q_j, q_j1, fraction)
                self.aligned_qw = np.append(self.aligned_qw, q_i.w)
                self.aligned_qx = np.append(self.aligned_qx, q_i.x)
                self.aligned_qy = np.append(self.aligned_qy, q_i.y)
                self.aligned_qz = np.append(self.aligned_qz, q_i.z)
                # IMU测量数据
                self.aligned_imu_wx = np.append(self.aligned_imu_wx, self.imu_raw_wx[i])
                self.aligned_imu_wy = np.append(self.aligned_imu_wy, self.imu_raw_wy[i])
                self.aligned_imu_wz = np.append(self.aligned_imu_wz, self.imu_raw_wz[i])
                self.aligned_imu_ax = np.append(self.aligned_imu_ax, self.imu_raw_ax[i])
                self.aligned_imu_ay = np.append(self.aligned_imu_ay, self.imu_raw_ay[i])
                self.aligned_imu_az = np.append(self.aligned_imu_az, self.imu_raw_az[i])
                # 此处不对 j 进行叠加，原因：两个gt时刻间可能有多个imu测量值
                i = i + 1 
        # 将 时间戳的单位由纳秒转为秒
        self.aligned_timestamp = self.aligned_timestamp * 1e-9

    def calculate_velocity(self):
        ### 直接进行一次差分求速度
        for i in range(np.size(self.aligned_px_m)-1):
            self.aligned_vx_mps = np.append( self.aligned_vx_mps, (self.aligned_px_m[i+1] - self.aligned_px_m[i])/(self.aligned_timestamp[i+1] - self.aligned_timestamp[i]))
            self.aligned_vy_mps = np.append( self.aligned_vy_mps, (self.aligned_py_m[i+1] - self.aligned_py_m[i])/(self.aligned_timestamp[i+1] - self.aligned_timestamp[i]))
            self.aligned_vz_mps = np.append( self.aligned_vz_mps, (self.aligned_pz_m[i+1] - self.aligned_pz_m[i])/(self.aligned_timestamp[i+1] - self.aligned_timestamp[i]))

        # 训练时需要避免使用最后一个时刻的数据
        self.aligned_vx_mps = np.append(self.aligned_vx_mps, self.aligned_vx_mps[-1])
        self.aligned_vy_mps = np.append(self.aligned_vy_mps, self.aligned_vy_mps[-1])
        self.aligned_vz_mps = np.append(self.aligned_vz_mps, self.aligned_vz_mps[-1])

        # 单位修正为 mps
        self.aligned_vx_mps = 1e9*self.aligned_vx_mps
        self.aligned_vy_mps = 1e9*self.aligned_vy_mps
        self.aligned_vz_mps = 1e9*self.aligned_vz_mps

        ### 此处用3点中值滤波对粗大误差进行了剔除
        signal.medfilt(self.aligned_vx_mps,3)
        signal.medfilt(self.aligned_vy_mps,3)
        signal.medfilt(self.aligned_vz_mps,3)

    def create_data_path(self, file_path_name):
        if os.path.exists(file_path_name):
            shutil.rmtree(file_path_name)  # 删除目录，包括目录下的所有文件
            os.mkdir(file_path_name)
        else:
            os.mkdir(file_path_name)

    def remove_bias(self):
        plt.figure(10,figsize=(10,10))   # 1表示figure的唯一编号，figsize表示图片大小
        plt.subplot(3,1,1)
        plt.plot(self.aligned_timestamp - self.aligned_timestamp[0], self.aligned_imu_wx, linewidth=.5, label='gyrox')
        plt.xlabel('time (s)')
        plt.subplot(3,1,2)
        plt.plot(self.aligned_timestamp - self.aligned_timestamp[0], self.aligned_imu_wy, linewidth=.5, label='gyrox')
        plt.xlabel('time (s)')
        plt.subplot(3,1,3)
        plt.plot(self.aligned_timestamp - self.aligned_timestamp[0], self.aligned_imu_wz, linewidth=.5, label='gyrox')
        plt.xlabel('time (s)')
        plt.show()

    def data_packet(self):
        # for SIZE_SET in size_set_list:
        for SIZE_SET in self.size_set_list:
            ### 将 aligned 数据写入文本 'w'模式表示写入，若文件已存在，则将其覆盖
            ### 完整的数据时序文件
            ### 数据分组
            ### 将test文件夹中的数据分成3组（6：2：2），分别放置于 train test val 文件夹中
            data_path = os.path.join(self.project_path, 'data/Euroc/')
            data_subdir = 'len_' + str(SIZE_SET) + '_' + self.relative_data_path.split('/')[-1] #'_MH01EASY/'
            train_ratio = 1.0
            test_ratio = 0.0
            val_ratio = 1 - train_ratio - test_ratio
            TOTAL_CNT = math.floor(np.size(self.aligned_timestamp)/SIZE_SET)
            # assert val_ratio >= 0 and val_ratio <= 1

            file_train_path = os.path.join(data_path, data_subdir, 'train')
            # file_test_path = os.path.join(data_path, data_subdir, 'test')
            # file_val_path = os.path.join(data_path, data_subdir, 'val')

            if not os.path.isdir(os.path.join(data_path, data_subdir)):
                os.mkdir(os.path.join(data_path, data_subdir))
            
            self.create_data_path(file_train_path)
            # self.create_data_path(file_test_path)
            # self.create_data_path(file_val_path)

            assert os.path.isdir(file_train_path)
            # assert os.path.isdir(file_test_path)
            # assert os.path.isdir(file_val_path)

            # with open(os.path.join(data_path, data_subdir) + 'data_aligned.txt', 'w', encoding = 'utf-8') as file_to_write:
            #     file_to_write.write('timestamp_ns,px_m,py_m,pz_m,vx_mps,vy_mps,vz_mps,qw,qx,qy,qz,gyrox_rps,gyroy_rps,gyroz_rps,accx_mps2,accy_mps2,accz_mps2\n')
            #     for i in range(np.size(self.aligned_timestamp)):
            #         file_to_write.write('{:.2f},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n'.format(
            #             self.aligned_timestamp[i],
            #             self.aligned_px_m[i],self.aligned_py_m[i],self.aligned_pz_m[i],
            #             self.aligned_vx_mps[i],self.aligned_vy_mps[i],self.aligned_vz_mps[i],
            #             self.aligned_qw[i],self.aligned_qx[i],self.aligned_qy[i],self.aligned_qz[i],
            #             self.aligned_imu_wx[i],self.aligned_imu_wy[i],self.aligned_imu_wz[i],
            #             self.aligned_imu_ax[i],self.aligned_imu_ay[i],self.aligned_imu_az[i]))

            tmp_set = np.empty([SIZE_SET, 11], dtype=np.float64)
            with tqdm(total=TOTAL_CNT) as pbar:
                for i in range(TOTAL_CNT):
                    pbar.update(1)
                    for j in range(SIZE_SET):
                        k = i*SIZE_SET + j
                        tmp_set[j,:] = np.array([
                            self.aligned_timestamp[k] - self.aligned_timestamp[0],
                            self.aligned_imu_wx[k],self.aligned_imu_wy[k],self.aligned_imu_wz[k],
                            self.aligned_imu_ax[k],self.aligned_imu_ay[k],self.aligned_imu_az[k],
                            self.aligned_qw[k],self.aligned_qx[k],self.aligned_qy[k],self.aligned_qz[k]
                            ], dtype=np.float64)
                        if j == SIZE_SET-1:
                            file_tmp_set = os.path.join(file_train_path, data_subdir[:-1]+str(i))
                            np.savez(file_tmp_set, tmp_set)


            # for root, dirs, files in os.walk(file_train_path):
            #     for file in files:
            #         idx_suffix = os.path.splitext(file)
            #         idx = int(idx_suffix[0])
            #         if idx <= TOTAL_CNT*train_ratio:
            #             continue
            #         elif idx > TOTAL_CNT*train_ratio and idx <= TOTAL_CNT*(train_ratio + test_ratio):
            #             shutil.move(os.path.join(root, file), os.path.join(file_test_path, file))
            #         else:
            #             shutil.move(os.path.join(root, file), os.path.join(file_val_path, file))

    

In [39]:
currentPath = os.getcwd().replace('\\','/')    # 获取当前路径 /home/sjtu/workspace/LWGC/code
project_path = os.path.abspath(os.path.join(currentPath, "..")) # /home/sjtu/workspace/LWGC

configs_path = os.path.join(project_path, 'configs/euroc_configs.yaml')
assert os.path.isfile(configs_path)

# 读取配置文件
with open(configs_path, 'r', encoding='utf8') as file:
    configs = yaml.safe_load(file)

for data_set in configs['relative_data_path']:
    # 实例化 euroc_analyser
    euroc_analyser = EurocAnalyser(project_path, data_set)

    # step1: 生成各个文件路径
    euroc_analyser.path_generator()

    # step2: 获取 size_set_list
    euroc_analyser.get_size_set_list()

    # step3: 获取 Euroc 数据集中位姿真值
    euroc_analyser.get_gt_pose()

    # step4: 获取 IMU 原始数据
    euroc_analyser.get_imu_raw_data()

    # step5: 时间戳对齐
    euroc_analyser.timestamp_align_old() # timestamp_align_v2()

    # step6: 根据对齐的数据差分求速度
    euroc_analyser.calculate_velocity()

    # 减去gyro bias
    euroc_analyser.remove_bias()

    # step7: 生成最终的数据包，用于后续训练
    ## 耗时较久，第一次生成后，可以将其注释
    euroc_analyser.data_packet()

    #########################################################################################
    ################################# 将数据进行归类 ##########################################
    #########################################################################################

    data_path = os.path.join(project_path, 'data/Euroc/')
    assert os.path.exists(data_path)

    # 遍历
    substr_len = 'len_'
    directories = [name for name in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, name))]
    print(directories)


    # 找到配置文件
    configs_path = os.path.join(project_path, 'configs/euroc_configs.yaml')
    assert os.path.isfile(configs_path)
    # 读取配置文件
    with open(configs_path, 'r', encoding='utf8') as file:
        configs = yaml.safe_load(file)
    size_set = configs['size_set_list']
    assert len(size_set) >= 1

    # 新建文件夹 Len_1200_Samples/train Len_1200_Samples/val

    train_path = os.path.join(data_path, 'Len_'+str(size_set[0])+'_Samples/train')
    val_path = os.path.join(data_path, 'Len_'+str(size_set[0])+'_Samples/val')
    if not os.path.exists(train_path):
        os.makedirs(train_path)
    if not os.path.exists(val_path):
        os.makedirs(val_path)

    # 将所有子文件夹的数据复制到 Len_1200_Samples/train 中
    destination_folder_for_train = train_path
    for directory in directories:
        if substr_len in directory:
            source_folder = os.path.join(data_path, directory, 'train')
            files_to_copy = [name for name in os.listdir(source_folder) if not os.path.isdir(os.path.join(source_folder, name))]
            for file_name in files_to_copy:
                source_file = os.path.join(source_folder, file_name)
                destination_file = os.path.join(destination_folder_for_train, file_name)
                shutil.copy2(source_file, destination_file)

    #########################################################################################
    #######################把数据按照设定的比例进行划分，并删除多余的中间数据########################
    #########################################################################################

    files_for_train = [name for name in os.listdir(destination_folder_for_train) if not os.path.isdir(os.path.join(destination_folder_for_train, name))]
    if 'V2_03_difficult' in euroc_analyser.relative_data_path:
        random.shuffle(files_for_train)
        # 将 train 文件夹中的部分数据抽取，作为 val 样本
        train_ratio = 0.7
        destination_folder_for_val = val_path
        idx = 0
        for file_name in files_for_train:
            idx = idx + 1
            if idx < len(files_for_train)*train_ratio:
                continue
            source_file = os.path.join(destination_folder_for_train, file_name)
            destination_file = os.path.join(destination_folder_for_val, file_name)
            shutil.move(source_file, destination_file)

        # 删除中间数据
        for directory in directories:
            if substr_len in directory:
                shutil.rmtree(os.path.join(data_path, directory))




RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 1000x1000 with 3 Axes>

100%|██████████| 57/57 [00:00<00:00, 768.47it/s]

['len_400_V2_03_difficult', 'V2_01_easy', 'MH_05_difficult', 'MH_04_difficult', 'MH_03_medium', 'MH_01_easy', 'V2_03_difficult', 'V1_02_medium', 'V2_02_medium', 'V1_03_difficult', 'MH_02_easy', 'V1_01_easy']
